# 07. RAG Parte 3: RetrievalQA

Agora vamos juntar as peças: Docs -> Split -> Vector Store -> Retriever -> LLM -> Resposta.

**Objetivos:**
- Criar uma `create_retrieval_chain` para responder perguntas baseadas nos documentos.

# Explicação Detalhada do Assunto

# 07. RAG Parte 3: RetrievalQA

Neste notebook, vamos consolidar o que aprendemos sobre Retrieval Augmented Generation (RAG) e construir um sistema completo e funcional. Uniremos as peças: documentos, divisão (splitting), armazenamento de vetores (vector store), recuperação (retriever), LLM (Large Language Model) e, finalmente, a resposta!

**Resumo Executivo:**

Este notebook marca o ponto culminante da nossa jornada de RAG. Implementaremos um sistema completo que recebe uma pergunta, busca informações relevantes em uma base de conhecimento, e utiliza um LLM para gerar uma resposta fundamentada. Aprenderemos a orquestrar todos os componentes do pipeline RAG, desde o carregamento e processamento dos documentos até a geração da resposta final.

**Conceitos Chave:**

*   **RAG (Retrieval Augmented Generation):** Uma técnica que combina a capacidade de recuperação de informações de um sistema de busca com a habilidade de geração de texto de um modelo de linguagem. Isso permite que o LLM responda perguntas com base em informações externas, em vez de depender apenas do seu conhecimento pré-existente.
*   **Chains (Correntes):** No contexto do LangChain, Chains são sequências de chamadas a componentes, como LLMs, prompts e utilitários. Elas permitem criar fluxos de trabalho complexos e automatizados. Neste notebook, usaremos chains para orquestrar o processo de recuperação e geração.
*   **Retriever (Recuperador):** Um componente responsável por buscar documentos relevantes em uma base de conhecimento, com base em uma consulta. Usaremos um retriever para encontrar os documentos mais relevantes para responder à pergunta do usuário.
*   **Vector Store (Armazenamento de Vetores):** Uma base de dados especializada em armazenar representações vetoriais de documentos. Isso permite realizar buscas semânticas eficientes, encontrando documentos que são semanticamente similares à consulta, mesmo que não compartilhem palavras-chave exatas.

**Objetivos de Aprendizado:**

Ao completar este notebook, você será capaz de:

*   Construir um pipeline RAG completo usando LangChain.
*   Implementar a função `create_retrieval_chain` para responder a perguntas com base em documentos recuperados.
*   Entender como os diferentes componentes do pipeline RAG interagem entre si.
*   Inspecionar as fontes de informação utilizadas para gerar uma resposta.
*   Adaptar e personalizar o pipeline RAG para diferentes casos de uso.

**Importância no Ecossistema LangChain:**

O RAG é uma das aplicações mais importantes e poderosas do LangChain. Ele permite construir sistemas que podem responder a perguntas complexas, gerar conteúdo personalizado e realizar tarefas de raciocínio, tudo com base em informações externas e atualizadas. Dominar o RAG é fundamental para construir aplicações de IA Generativa robustas e úteis.

Vamos começar!

---


In [1]:
### INJECTION START ###
import os
from dotenv import load_dotenv
import sys
# Carrega .env do local ou de pastas comuns
for p in ['.', '..', 'scripts', '../scripts']:
    path = os.path.join(p, '.env')
    if os.path.exists(path):
        load_dotenv(path)
        break
if os.getenv('GOOGLE_API_KEY'):
    os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY')
    os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY')
### INJECTION END ###

import os
from dotenv import load_dotenv
import sys
# Autenticação automática do script
for p in ['.', '..', 'scripts', '../scripts']:
    path = os.path.join(p, '.env')
    if os.path.exists(path):
        load_dotenv(path)
        break
if os.getenv('GOOGLE_API_KEY'):
    os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY')

import os
from dotenv import load_dotenv
load_dotenv()

# !pip install -qU langchain langchain-openai langchain-community faiss-cpu python-dotenv # Script-patched

False

In [2]:
import os
try:
    from google.colab import userdata
except ImportError:
    userdata = None
import getpass

try:
    pass # Script-patched: using env var
except:
    pass # Script-patched: using env var

## 1. Setup Rápido (Load, Split, Index)

Recriando o índice para usar aqui.

In [3]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS

# 1. Load
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
docs = loader.load()

# 2. Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# 3. Index
vectorstore = FAISS.from_documents(splits, GoogleGenerativeAIEmbeddings())
retriever = vectorstore.as_retriever()

/Users/naubergois/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
USER_AGENT environment variable not set, consider setting it to identify your requests.


/Users/naubergois/Library/Python/3.9/lib/python/site-packages/google/api_core/_python_version_support.py:246: FutureWarning: You are using a non-supported Python version (3.9.6). Google will not post any further updates to google.api_core supporting this Python version. Please upgrade to the latest Python version, or at least Python 3.10, and then update google.api_core.
  warnings.warn(message, FutureWarning)
/Users/naubergois/Library/Python/3.9/lib/python/site-packages/google/auth/__init__.py:54: FutureWarning: You are using a Python version 3.9 past its end of life. Google will update google-auth with critical bug fixes on a best-effort basis, but not with any other fixes or features. Please upgrade your Python version, and then update google-auth.
  warnings.warn(eol_message.format("3.9"), FutureWarning)
/Users/naubergois/Library/Python/3.9/lib/python/site-packages/google/oauth2/__init__.py:40: FutureWarning: You are using a Python version 3.9 past its end of life. Google will upda

/Users/naubergois/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/naubergois/Library/Python/3.9/lib/python/site-packages/langchain_google_genai/chat_models.py:47: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  from google.generativeai.caching import CachedContent  # type: ignore[import]


ValidationError: 1 validation error for GoogleGenerativeAIEmbeddings
model
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing

## 2. Criando a Chain de RAG

Usaremos `create_stuff_documents_chain` (que insere os docs no prompt) e `create_retrieval_chain` (que gerencia a busca).

In [4]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)

# Prompt do sistema que receberá o contexto
system_prompt = (
    "Você é um assistente para tarefas de perguntas e respostas. "
    "Use os seguintes pedaços de contexto recuperado para responder à pergunta. "
    "Se você não souber a resposta, diga que não sabe. "
    "Use no máximo três frases e mantenha a resposta concisa."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

# Chain que combina documentos no prompt
question_answer_chain = create_stuff_documents_chain(llm, prompt)

# Chain final que recupera docs e passa para a chain acima
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

NameError: name 'retriever' is not defined

## 3. Testando

Vamos fazer uma pergunta sobre o artigo.

In [5]:
response = rag_chain.invoke({"input": "What is Task Decomposition?"})

print(response["answer"])

NameError: name 'rag_chain' is not defined

## 4. Inspecionando a Fonte

Podemos ver quais documentos foram usados para gerar a resposta.

In [6]:
for i, doc in enumerate(response["context"]):
    print(f"Documento {i}: {doc.page_content[:100]}...")

NameError: name 'response' is not defined

## Conclusão

Temos um sistema de RAG funcional! Ele recupera informação relevante e responde de forma fundamentada.

No próximo notebook, vamos sair do padrão "pergunta-resposta" e entrar no mundo dos **Agentes**, que podem usar ferramentas para agir.